# First project

In [1]:
import pandas as pd
import warnings
import tkinter as tk

warnings.filterwarnings('ignore')

In [2]:
cols=('used id','item_id','rating','timestamp')
df=pd.read_csv('u.data.zip',sep='\t',names=cols)
cols_movies=['item_id','title']+[str(i) for i in range(22)]
movie_titles=pd.read_csv('u.item',sep='|',names=cols_movies)
movie_titles=movie_titles[['item_id','title']]
data=pd.merge(df,movie_titles, on="item_id")
rating=pd.DataFrame(data.groupby('title')['rating'].mean())
rating['count']=data['title'].value_counts()
pivot_df=data.pivot_table(index='used id',columns='title',values='rating')



app= tk.Tk()
app.title("Recommendation System")
app.geometry("400x300")
app.iconbitmap('depositphotos_90102796_stock_illustration_cinema_film_clapper_board_vector___copy_rti_icon.ico')
app.configure(bg='light blue')


tk.Label(app,text="which movie you watched:",bg='light blue',font=('Sitka Heading', 10, 'bold')).place(x=10,y=30)
tk.Label(app,text="Recommended for you:",bg='light blue',font=('Sitka Heading', 10, 'bold')).place(x=10,y=140)
tk.Label(app,text="Also try:",bg='light blue',font=('Sitka Heading', 10, 'bold')).place(x=10,y=160)

movie_var=tk.Variable()
tk.Entry(app,textvariable=movie_var,width=30).place(x=180,y=30)

Recommend_var=tk.Variable(app)
Recommend_var.set("type movie name...")
tk.Label(app,textvariable=Recommend_var,bg='light blue',font=('Sitka Heading', 10)).place(x=150,y=140)

also_try_var=tk.Variable(app)
also_try_var.set("type movie name...")
tk.Label(app,textvariable=also_try_var,bg='light blue',font=('Sitka Heading', 10)).place(x=90,y=160)

def find_recommendation():
    movie= movie_var.get().lower().strip()
    try:
        movie=movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]
        movie_var.set(movie)
    except IndexError:
        Recommend_var.set('movie not found')
        also_try_var.set('movie not found')
    else:
        corr_df=pd.DataFrame(pivot_df.corrwith(pivot_df[movie]),columns=['correlation'])
        corr_df.dropna(inplace=True)
        corr_df=corr_df.join(rating['count'])
        Recommend_var.set(corr_df[(corr_df['count']>200)&(corr_df['correlation']>0.4)].sort_values(by='correlation',ascending=False).index[1])
        also_try_var.set(corr_df[corr_df['correlation']>0.4].sort_values(by='correlation',ascending=False).index[0])

tk.Button(app,text='Find suggestion',command=find_recommendation,bg="black",fg='#ffffff').place(x=125,y=75)
app.mainloop()